In [52]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from fancyimpute import KNN
from sklearn.metrics import mean_squared_error
from math import sqrt
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
from pmdarima import auto_arima
import sys
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose 
from statsmodels.tsa.stattools import adfuller
from openpyxl import load_workbook
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFECV
from xgboost import XGBRegressor
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

In [53]:
def get_mse1(X,y,method):
    X_train = X.iloc[:774,:]
    X_test = X.iloc[774:,:]
    y_train = y.iloc[:774]
    y_test = y.iloc[774:]
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    model = LinearRegression()
    model.fit(method.fit_transform(X_train, y_train), y_train)
    y_pred = model.predict(method.transform(X_test))
    mse = mean_squared_error(y_test, y_pred)
    return mse

In [54]:
df = pd.read_excel(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx", sheet_name='Version4',engine='openpyxl',index_col='Date',parse_dates=True)
df.head()

,Money Market - Net liquidity injected,Money Market - Call Money,Money Market - Triparty Money,Foreign Exchange Turnover (Purchases) - Merchant FCY/INR - Spot,Foreign Exchange Turnover (Purchases) -Merchnat FCY/INR - Forward,Foreign Exchange Turnover (Purchases) -Merchant FCY/INR - Forward Cancel,Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Spot,Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Forward,Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Forward Cancel,Foreign Exchange Turnover (Purchases) -Inter Bank FCY/INR - Spot,...,NSDL FPI - EQ_PMO,NSDL FPI - D_SE,NSDL FPI - D-PMO,NSDL FPI - D-ST,NSDL FPI - DVRR_SE,NSDL FPI - DVRR-PMO,NSDL FPI - H_SE,NSDL FPI - H-PMO,NSDL FPI - Total,Value of Rupee
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-14,-0.179645,-0.140530,1.888865,2297.000000,861.000000,488.000000,47.000000,89,36.000000,6402.000000,...,-55.690000,453.93,-0.210000,453.720000,-0.21,41.77,0.650000,0.0,588.68,71.45
2020-02-15,-0.181037,-0.066684,2.466285,1336.838723,586.685775,292.363524,-42.635685,44,-5.862873,2779.851583,...,-17.166667,453.93,8.755250,524.714736,0.00,0.00,1.223889,0.0,25.64,71.45
2020-02-16,-0.152035,0.078441,2.463705,671.660449,340.320116,113.026919,-65.615348,44,2.857913,2562.644568,...,21.356667,453.93,6.797701,391.039830,0.00,0.00,1.008416,0.0,25.64,71.45
2020-02-17,-0.044363,-0.055662,2.461125,1714.000000,509.000000,356.000000,60.000000,44,35.000000,6688.000000,...,59.880000,182.83,2.240000,185.070000,0.00,0.00,0.550000,0.0,25.64,71.38
2020-02-18,0.068265,-0.017421,2.498545,3017.000000,739.000000,735.000000,210.000000,61,39.000000,9594.000000,...,-49.650000,-15.97,-0.610000,-16.580000,0.00,5.60,0.330000,0.0,13.61,71.55


In [72]:
models = {}

In [73]:
X = df.drop(columns=['Value of Rupee'])
y = df['Value of Rupee']

In [76]:
mi = {}
for k in range(2,50):
    selector = SelectKBest(mutual_info_regression, k=k)
    selector.fit(X, y)
    mse = get_mse1(X,y,selector)
    mi['Mutual Info Regression :- Value of k = '+str(k)] = mse
sorted_mi = dict(sorted(mi.items(), key=lambda item: item[1]))
models[list(sorted_mi.keys())[0]] = sorted_mi[list(sorted_mi.keys())[0]]
sorted_mi

{'Mutual Info Regression :- Value of k = 10': 28.40280765346802,
 'Mutual Info Regression :- Value of k = 9': 28.976589587507483,
 'Mutual Info Regression :- Value of k = 11': 29.029962650994577,
 'Mutual Info Regression :- Value of k = 8': 29.572353136696336,
 'Mutual Info Regression :- Value of k = 7': 29.815354527216936,
 'Mutual Info Regression :- Value of k = 12': 30.184467876830585,
 'Mutual Info Regression :- Value of k = 27': 30.913513532168178,
 'Mutual Info Regression :- Value of k = 25': 30.922942980787816,
 'Mutual Info Regression :- Value of k = 13': 30.95487231008614,
 'Mutual Info Regression :- Value of k = 26': 30.982794309251073,
 'Mutual Info Regression :- Value of k = 28': 31.00631378670237,
 'Mutual Info Regression :- Value of k = 24': 31.01448636645646,
 'Mutual Info Regression :- Value of k = 29': 31.04057231477621,
 'Mutual Info Regression :- Value of k = 14': 31.373115362680792,
 'Mutual Info Regression :- Value of k = 17': 31.40616864424293,
 'Mutual Info Regre

In [78]:
pca_values ={}
for n in range(2,50):
    pca = PCA(n_components=n)
    pca.fit(X)
    mse = get_mse1(X,y,pca)
    pca_values['PCA n_components = '+str(n)] = mse
sorted_pca_values = dict(sorted(pca_values.items(), key=lambda item: item[1]))
models[list(sorted_pca_values.keys())[0]] = sorted_pca_values[list(sorted_pca_values.keys())[0]]
sorted_pca_values

{'PCA n_components = 49': 36.13399825808781,
 'PCA n_components = 48': 37.58272947657567,
 'PCA n_components = 2': 38.26612819218628,
 'PCA n_components = 11': 38.3234398042522,
 'PCA n_components = 3': 38.41088734009764,
 'PCA n_components = 4': 38.446068051692315,
 'PCA n_components = 5': 38.50617898832683,
 'PCA n_components = 6': 38.52428927682254,
 'PCA n_components = 8': 38.53071604365728,
 'PCA n_components = 47': 38.63228637919133,
 'PCA n_components = 9': 38.718824306566866,
 'PCA n_components = 12': 38.750192804525234,
 'PCA n_components = 7': 38.764356251946474,
 'PCA n_components = 14': 39.305202491624534,
 'PCA n_components = 10': 39.3328316911189,
 'PCA n_components = 13': 39.50194881883341,
 'PCA n_components = 29': 39.628378442736754,
 'PCA n_components = 17': 39.63299998345309,
 'PCA n_components = 18': 39.676868045927904,
 'PCA n_components = 22': 39.78058626138,
 'PCA n_components = 44': 39.79665808506255,
 'PCA n_components = 23': 39.80650069610827,
 'PCA n_componen

In [81]:
lasso = Lasso()
X_train = X.iloc[:774,:]
X_test = X.iloc[774:,:]
y_train = y.iloc[:774]
y_test = y.iloc[774:]
alpha_range = np.logspace(-5, 1, 100)
param_dist = {'alpha': alpha_range}
random_search = RandomizedSearchCV(lasso, param_distributions=param_dist, cv=5)
random_search.fit(X, y)
print("Best alpha value: ", random_search.best_params_['alpha'])
best_lasso = Lasso(alpha=random_search.best_params_['alpha'])
best_lasso.fit(X_train, y_train)
lasso_preds = best_lasso.predict(X_test)
lasso_mse = mean_squared_error(y_test, lasso_preds)
selected_features = X.columns[best_lasso.coef_ != 0]
models['Lasso Regression'] = lasso_mse
print("Selected features: ", selected_features)
print("Lasso MSE: ", lasso_mse)

Best alpha value:  8.697490026177835
Selected features:  Index(['Foreign Exchange Turnover (Purchases) - Merchant FCY/INR - Spot',
       'Foreign Exchange Turnover (Purchases) -Merchnat FCY/INR - Forward',
       'Foreign Exchange Turnover (Purchases) -Merchant FCY/INR - Forward Cancel',
       'Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Spot',
       'Foreign Exchange Turnover (Purchases) -Inter Bank FCY/INR - Spot',
       'Foreign Exchange Turnover (Purchases) -Inter Bank FCY/INR - Swap',
       'Foreign Exchange Turnover (Purchases) -Inter Bank FCY/INR - Forward',
       'Foreign Exchange Turnover (Purchases) -Inter Bank FCY/FCY - Swap',
       'Foreign Exchange Turnover (Sales) -Merchant FCY/INR - Spot',
       'Foreign Exchange Turnover (Sales) -Merchant FCY/INR - Forward Cancel',
       'Outward Remittances', 'Foreign Investment Inflows',
       'Foreign Trade - Trade Balance', 'Foreign Trade - Trade Balance - Oil',
       'Maturity Breakdown - Total',
       'Sa

In [83]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

# define ridge regression model
ridge = Ridge()

# define the grid of hyperparameters to search over
param_distributions = {'alpha': np.logspace(-5, 2, 50)}

# define the randomized search cv object
random_search = RandomizedSearchCV(ridge, param_distributions=param_distributions, cv=5, n_iter=50, random_state=42)

# perform the randomized search
random_search.fit(X, y)

# print the best hyperparameters and the corresponding score
print("Best hyperparameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)

# get the best model
best_model = random_search.best_estimator_

# get the coefficients of the best model
coef = best_model.coef_

# define the range of number of features to consider
num_features = range(1, X.shape[1]+1)

# initialize list to store cross-validation scores
cv_scores = []

# loop over different number of features
for k in num_features:
    # get the indices of the top k features
    top_features = np.argsort(np.abs(coef))[::-1][:k]

    # get the top k features from the data
    X_top = X[:, top_features]

    # perform cross-validation with 5 folds
    scores = cross_val_score(best_model, X_top, y, cv=5)

    # store the mean cross-validation score for the current k
    cv_scores.append(np.mean(scores))

# find the optimal number of features
optimal_num_features = num_features[np.argmax(cv_scores)]

# get the indices of the top optimal_num_features features
top_features = np.argsort(np.abs(coef))[::-1][:optimal_num_features]

# get the names of the top optimal_num_features features
feature_names = ['Feature ' + str(i+1) for i in top_features]
features = []
for i in list(top_features):
    features.append(df.columns[i])


Best hyperparameters:  {'alpha': 100.0}
Best score:  -11.56474867982914


In [84]:
X = df.drop(columns=['Value of Rupee'])
y = df['Value of Rupee']
X_train = X.iloc[:774,:]
X_test = X.iloc[774:,:]
y_train = y.iloc[:774]
y_test = y.iloc[774:]
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
best_model.fit(X_train,y_train)
preds = best_model.predict(X_test)
mse = mean_squared_error(y_test, preds)
models['Ridge Regression'] = mse
mse

38.62244804130635

In [86]:
models = dict(sorted(models.items(), key=lambda item: item[1]))
models

{'Mutual Info Regression :- Value of k = 10': 28.40280765346802,
 'Lasso Regression': 31.05013824647375,
 'PCA n_components = 49': 36.13399825808781,
 'Ridge Regression': 38.62244804130635}

In [108]:
X = df.drop(columns=['Value of Rupee'])
y = df['Value of Rupee']
selector = SelectKBest(mutual_info_regression, k=10)
X_new = selector.fit_transform(X, y)
selected_features_indices = selector.get_support(indices=True)
selected_features_mi = X.columns[selected_features_indices]
selected_features_mi = list(selected_features_mi)
selected_features_mi.append('Value of Rupee')
print("Selected features: ", selected_features_mi)
df2 = df[selected_features_mi]

Selected features:  ['Outward Remittances', 'Foreign Investment Inflows', 'Foreign Trade - Trade Balance - Oil', 'Sales and Purchases of Dollar -Sale ', 'Sales and Purchases of Dollar -Net', 'International Trade Services - Imports', 'International Trade Services - Exports', 'External Commercial borrowings - Total Number', 'External Commercial borrowings - Total Amount', 'External Commercial borrowings - Weighted Average Maturity', 'Value of Rupee']


In [110]:
selected_features_lasso = X.columns[best_lasso.coef_ != 0]
selected_features_lasso = list(selected_features_lasso)
selected_features_lasso.append('Value of Rupee')
print(selected_features_lasso)
df3 = df[selected_features_lasso]

['Foreign Exchange Turnover (Purchases) - Merchant FCY/INR - Spot', 'Foreign Exchange Turnover (Purchases) -Merchnat FCY/INR - Forward', 'Foreign Exchange Turnover (Purchases) -Merchant FCY/INR - Forward Cancel', 'Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Spot', 'Foreign Exchange Turnover (Purchases) -Inter Bank FCY/INR - Spot', 'Foreign Exchange Turnover (Purchases) -Inter Bank FCY/INR - Swap', 'Foreign Exchange Turnover (Purchases) -Inter Bank FCY/INR - Forward', 'Foreign Exchange Turnover (Purchases) -Inter Bank FCY/FCY - Swap', 'Foreign Exchange Turnover (Sales) -Merchant FCY/INR - Spot', 'Foreign Exchange Turnover (Sales) -Merchant FCY/INR - Forward Cancel', 'Outward Remittances', 'Foreign Investment Inflows', 'Foreign Trade - Trade Balance', 'Foreign Trade - Trade Balance - Oil', 'Maturity Breakdown - Total', 'Sales and Purchases of Dollar - Purchase', 'Sales and Purchases of Dollar -Sale ', 'Sales and Purchases of Dollar -Net', 'International Trade Services - Imp

In [112]:
import pandas as pd
from openpyxl import load_workbook
book = load_workbook(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")
writer = pd.ExcelWriter(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx", engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
sheet_name = 'Version5.1'
df2.index = df2.index.strftime('%d-%m-%Y')
df2.to_excel(writer, sheet_name=sheet_name, index=True)
writer.save()
book.save(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")

In [113]:
import pandas as pd
from openpyxl import load_workbook
book = load_workbook(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")
writer = pd.ExcelWriter(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx", engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
sheet_name = 'Version5.2'
df3.index = df3.index.strftime('%d-%m-%Y')
df3.to_excel(writer, sheet_name=sheet_name, index=True)
writer.save()
book.save(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")

In [114]:
df4 = pd.DataFrame.from_dict(models,orient='index').reset_index()
df4.columns = ['Best Models','MSE']
df4.set_index('Best Models')
book = load_workbook(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")
writer = pd.ExcelWriter(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx", engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
sheet_name = 'Report2 - Version5'
df4.to_excel(writer, sheet_name=sheet_name, index=True,startrow=3, startcol=0)
writer.save()
book.save(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")

In [115]:
df5 = pd.DataFrame.from_dict(sorted_mi,orient='index').reset_index()
df5.columns = ['Mutual info Regression Models','MSE']
df5.set_index('Mutual info Regression Models')
book = load_workbook(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")
writer = pd.ExcelWriter(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx", engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
sheet_name = 'Report1 - Version5'
df5.to_excel(writer, sheet_name=sheet_name, index=True,startrow=3, startcol=0)
writer.save()
book.save(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")

In [116]:
df6 = pd.DataFrame.from_dict(sorted_pca_values,orient='index').reset_index()
df6.columns = ['PCA Models','MSE']
df6.set_index('PCA Models')
book = load_workbook(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")
writer = pd.ExcelWriter(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx", engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
sheet_name = 'Report1 - Version5'
df6.to_excel(writer, sheet_name=sheet_name, index=True,startrow=3, startcol=5)
writer.save()
book.save(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")